## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

C:\Users\Bill\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [4]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(filters=32, kernel_size=(3,3), input_shape=(32,32,3), activation='relu', padding='same'))
#32,3,3,input_shape=(32,32,3),activation='relu'
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(filters=32, kernel_size=(3,3), padding='same'))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(units=100, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(units=10, activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)









Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
50000/50000 [==============================] - 8s 155us/step - loss: 1.3619 - acc: 0.5319
Epoch 2/100
50000/50000 [==============================] - 6s 119us/step - loss: 0.9455 - acc: 0.6677
Epoch 3/100
50000/50000 [==============================] - 6s 126us/step - loss: 0.7577 - acc: 0.7340
Epoch 4/100
50000/50000 [==============================] - 6s 125us/step - loss: 0.6135 - acc: 0.7844
Epoch 5/100
50000/50000 [==============================] - 6s 126us/step - loss: 0.4797 - acc: 0.8314
Epoch 6/100
50000/50000 [==============================] - 6s 122us/step - loss: 0.3762 - acc: 0.8687
Epoch 7/100
50000/50000 [==============================] - 6s 122us/step - loss: 0.2882 - acc: 0.8996
Epoch 8/100
50000/50000 [==============================] - 6s 120us/step - loss: 0.2303 - acc: 0.9199
Epoch 9/100
50000/50000 [==============================] - 6s 119us/step - loss: 0

50000/50000 [==============================] - 6s 118us/step - loss: 0.0198 - acc: 0.9940
Epoch 62/100
50000/50000 [==============================] - 6s 117us/step - loss: 0.0227 - acc: 0.9930
Epoch 63/100
50000/50000 [==============================] - 6s 117us/step - loss: 0.0301 - acc: 0.9902
Epoch 64/100
50000/50000 [==============================] - 6s 117us/step - loss: 0.0268 - acc: 0.9913 
Epoch 65/100
50000/50000 [==============================] - 6s 117us/step - loss: 0.0253 - acc: 0.9919
Epoch 66/100
50000/50000 [==============================] - 6s 117us/step - loss: 0.0172 - acc: 0.9943
Epoch 67/100
50000/50000 [==============================] - 6s 118us/step - loss: 0.0265 - acc: 0.9917
Epoch 68/100
50000/50000 [==============================] - 6s 119us/step - loss: 0.0217 - acc: 0.9932
Epoch 69/100
50000/50000 [==============================] - 6s 117us/step - loss: 0.0316 - acc: 0.9903
Epoch 70/100
50000/50000 [==============================] - 6s 118us/step - loss: 0.0

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[3.4196633e-05, 2.4186542e-09, 3.5899319e-02, 9.6212995e-01,
        1.9361140e-03, 4.5821542e-09, 5.6538497e-08, 1.2245353e-11,
        7.9422428e-08, 2.4543795e-07]], dtype=float32)